<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1q31zNNG1toRhcpoSnTpllZtTw0C6HwOJ?usp=sharing)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

## **Unsloth: Fine-tune Models 2-5x Faster with 80% Less Memory 🚀**  
Unsloth optimizes the fine-tuning of models like Llama 3.2, Mistral, Phi-3.5, Qwen 2.5, and Gemma, achieving up to 5x faster training.  
It reduces memory usage by 80%, enabling efficient training even on hardware with limited resources.


###**Setup and Installation**

In [ ]:
pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git" datasets evaluate unsloth

In [ ]:
!pip uninstall -y torchvision
!pip install torchvision

### **Install and Import Necessary Libraries 📦**


In [ ]:
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### **Set Up Dataset and Max Sequence Length 📊**

In [ ]:
max_seq_length = 2048 # Supports RoPE Scaling internally, so choose any!
# Get LAION dataset
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files = {"train" : url}, split = "train")


### **Define Available 4-bit Quantized Models ⚡**

In [ ]:
# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
] # More models at https://huggingface.co/unsloth


### **Load Pretrained Model with 4-bit Quantization 🔑**
We are Using llama-3-8b Model for Quantization


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

### **Apply LoRA Fine-Tuning to the Model 🔧**


In [ ]:
# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### **Set Up the Trainer and Training Arguments 🏋️**


In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()


### **Prepare the Model for Inference and Generate Text 🤖**


In [ ]:
# Ensure you're using CUDA (GPU) if available, otherwise fallback to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the selected device (CUDA or CPU)
model = model.to(device)

###Query the Model

In [ ]:
input_text = "Tell me a story about artificial intelligence and ethics."
outputs = model.generate(**inputs, max_length=200)

# Decode and print the result
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Tell me a story about artificial intelligence and ethics. What's the story?
Artificial intelligence (AI) is a rapidly growing field of research and development. It is a branch of computer science that deals with the creation of intelligent machines that are capable of performing tasks that would normally require human intelligence. AI is used in a variety of fields, including robotics, natural language processing, computer vision, and machine learning. AI is also used in a variety of applications, such as autonomous vehicles, virtual assistants, and medical diagnosis. AI is a powerful tool, and it has the potential to revolutionize the way we live and work. However, it is important to consider the ethical implications of AI. AI can be used for both good and bad purposes, and it is important to ensure that AI is used responsibly and ethically. AI should be used to benefit humanity, and not to harm it. AI should be used to improve our lives, and not to replace them. AI should be used to 